### 一、准备数据

imdb数据集的目标是根据电影评论的文本内容预测评论的情感标签。

训练集有20000条电影评论文本，测试集有5000条电影评论文本，其中正面评论和负面评论都各占一半。

文本数据预处理较为繁琐，包括中文切词（本示例不涉及），构建词典，编码转换，序列填充，构建数据管道等等。

在tensorflow中完成文本数据预处理的常用方案有两种，

- 第一种是利用 **tf.keras.preprocessing** 中的 **Tokenizer** 词典构建工具和 **tf.keras.utils.Sequence** 构建文本数据生成器管道。

- 第二种是使用 **tf.data.Dataset** 搭配 **tf.keras.layers.experimental.preprocessing.TextVectorization**预处理层。

第二种方法为TensorFlow原生方式，相对也更加简单一些。

我们此处介绍第二种方法。

![电影评论](../data/电影评论.jpg)

#### 1.1 构建管道-构建预处理

In [1]:
import tensorflow as tf

In [14]:
def split_line(line):
    """ 处理每一行样本
    
    Parameters:
    ----------
    line: string
        每行样本的格式 样本标签 制表符 样本文本
    
    Returns:
    ----------
    text: tf.Tensor, dtype=string, shape=(1,)
        样本文本
    label: tf.Tensor, dtype=int32, shape=(1,)
        样本标签
    """
    arr = tf.strings.split(line,"\t")
    label = tf.expand_dims(tf.cast(tf.strings.to_number(arr[0]),tf.int32),axis = 0)
    text = tf.expand_dims(arr[1],axis = 0)
    return (text,label)

**tf.strings.split: 字符串分词**

- [简化NLP：TensorFlow中tf.strings的使用](https://zhuanlan.zhihu.com/p/98718689)

**tf.strings.to_number:将输入张量中的每个字符串转换为指定的数值类型**
- [TensorFlow函数：tf.string_to_number](https://www.w3cschool.cn/tensorflow_python/tensorflow_python-2kdb2my1.html)

**tf.expand_dims: 扩展维度**
- [tf.expand_dims](https://www.cnblogs.com/helloworld0604/p/9001703.html)

In [17]:
train_data_path = "../data/imdb/train.csv"
test_data_path =  "../data/imdb/test.csv"

BATCH_SIZE = 20 

ds_train_raw =  tf.data.TextLineDataset(filenames = [train_data_path]) \
   .map(split_line,num_parallel_calls = tf.data.experimental.AUTOTUNE) \
   .shuffle(buffer_size = 1000).batch(BATCH_SIZE) \
   .prefetch(tf.data.experimental.AUTOTUNE)

ds_test_raw = tf.data.TextLineDataset(filenames = [test_data_path]) \
   .map(split_line,num_parallel_calls = tf.data.experimental.AUTOTUNE) \
   .batch(BATCH_SIZE) \
   .prefetch(tf.data.experimental.AUTOTUNE)

待添加注释

#### 1.2 构建词典

In [31]:
def clean_text(text):
    lowercase = tf.strings.lower(text)
    stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
    cleaned_punctuation = tf.strings.regex_replace(stripped_html,
         '[%s]' % re.escape(string.punctuation),'')
    return cleaned_punctuation

In [33]:

vectorize_layer = TextVectorization(
    standardize=clean_text,
    split = 'whitespace',
    max_tokens=MAX_WORDS-1, #有一个留给占位符
    output_mode='int',
    output_sequence_length=MAX_LEN)

NameError: name 'TextVectorization' is not defined